In [1]:
from tsdm.datasets import Electricity

In [2]:
print(dir(Electricity))

In [3]:
print(dir(Electricity("")))

In [4]:
Electricity.url


In [5]:
Electricity.dataset

In [6]:
%%timeit -n 1 -r 1
_ = Electricity.dataset

In [7]:
%%timeit -n 1 -r 100
_ = Electricity.dataset